#### write a function that takes the data and exports/saves numpy files of encoded data

In [1]:
import time
beginning_time = time.time()

import sys
sys.path.append("../Code/")
from utils import performance

import os
DATA_DIR = os.path.join('..', 'Data')

import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
# give the option for practice mode (i.e. don't load the full dataset)
practice = False

# load data
if practice:
    train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), nrows=1000)
    valid = pd.read_csv(os.path.join(DATA_DIR, 'validation.csv'), nrows=1000)
    test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'), nrows=1000)
else:
    train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
    valid = pd.read_csv(os.path.join(DATA_DIR, 'validation.csv'))
    test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [4]:
train_X = train.drop(['click','bidprice', 'payprice'], axis=1)
train_y = train[['click','bidprice', 'payprice']].copy()

valid_X = valid.drop(['click','bidprice', 'payprice'], axis=1)
valid_y = valid[['click','bidprice', 'payprice']].copy()

del train, valid

In [5]:
test_X = test

del test

In [6]:
# columns to be removed
remove = ["bidid", "userid", "IP", "url", "urlid"]

train_X = train_X.drop(remove, axis=1)
valid_X = valid_X.drop(remove, axis=1)
test_X = test_X.drop(remove, axis=1)

In [7]:
# fix usertag (from lists to 1 hot encode)
train_X.usertag = train_X.usertag.apply(lambda x: x.split(","))
valid_X.usertag = valid_X.usertag.apply(lambda x: x.split(","))
test_X.usertag = test_X.usertag.apply(lambda x: x.split(","))

In [8]:
sss= set()
for i in train_X.usertag:
    sss |= set(i)
print("there are {} usertags in total train set".format(len(sss)))

there are 69 usertags in total train set


In [9]:
ttt= set()
for i in valid_X.usertag:
    ttt |= set(i)
print("there are {} usertags in total valid set".format(len(ttt)))

there are 69 usertags in total valid set


In [10]:
uuu= set()
for i in test_X.usertag:
    uuu |= set(i)
print("there are {} usertags in total test set".format(len(uuu)))

there are 69 usertags in total test set


In [11]:
len(sss-ttt)

0

In [12]:
len(ttt-uuu)

0

In [13]:
# slotID and domain are too big to get dummies for 

#-> instead keep the top 100 from each

from collections import defaultdict as dd

slid = dd(int)
dom = dd(int)

for x in train_X.slotid:
    slid[x]+=1
    
for x in train_X.domain:
    dom[x]+=1
    
    
n = 5000

# there are too many "domain" and "slotid"
# we only keep the ones with frequency over 5000 in the training set

keep_slotid = set()
keep_domain = set()

for a,b in slid.items():
    if b>5000:
        keep_slotid |= {a}
        
for a,b in dom.items():
    if b>5000:
        keep_domain |= {a}

In [14]:
len(keep_domain)

63

In [15]:
len(keep_slotid)

71

In [16]:
def my_map(x, S):
    if x in S:
        return(x)
    else:
        return("null")

train_X.slotid = train_X.slotid.apply(lambda x: my_map(x, keep_slotid))
valid_X.slotid = valid_X.slotid.apply(lambda x: my_map(x, keep_slotid))
test_X.slotid = test_X.slotid.apply(lambda x: my_map(x, keep_slotid))

train_X.domain = train_X.domain.apply(lambda x: my_map(x, keep_domain))
valid_X.domain = valid_X.domain.apply(lambda x: my_map(x, keep_domain))
test_X.domain = test_X.domain.apply(lambda x: my_map(x, keep_domain))

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

train_X = train_X.join(pd.DataFrame(mlb.fit_transform(train_X.pop('usertag')),
                          columns=mlb.classes_,
                          index=train_X.index))

In [18]:
valid_X = valid_X.join(pd.DataFrame(mlb.fit_transform(valid_X.pop('usertag')),
                          columns=mlb.classes_,
                          index=valid_X.index))

In [19]:
test_X = test_X.join(pd.DataFrame(mlb.fit_transform(test_X.pop('usertag')),
                          columns=mlb.classes_,
                          index=test_X.index))

In [20]:
train_X.columns==valid_X.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [21]:
train_X.columns==test_X.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [22]:
len(train_X.columns)

85

In [23]:
len(valid_X.columns)

85

In [24]:
len(test_X.columns)

85

In [25]:
time.time()-beginning_time

63.28274393081665

In [26]:
# columns to get dummies for (the others make more sense to keep asis)
dummy = ["useragent", "region", "city", "adexchange", "domain", "slotid", "slotvisibility", "slotformat", 
        "creative", "keypage"]

#dummy = ["useragent", "region", "city", "adexchange", "slotvisibility", "slotformat"]

In [27]:
train_X = pd.get_dummies(train_X, columns=dummy)#, sparse=True)

In [28]:
len(train_X.columns)

823

In [29]:
time.time()-beginning_time

92.68676018714905

In [30]:
test_X = pd.get_dummies(test_X, columns=dummy)#, sparse=True)
valid_X = pd.get_dummies(valid_X, columns=dummy)#, sparse=True)

In [31]:
time.time()-beginning_time

98.11624121665955

In [32]:
sys.getsizeof(train_X)/1000000000

3.276962492

In [33]:
# make sure columns are aligned across the three sets

# need to clean up the validation set so it is consistent with the training set


#-> remove additional columns
drop_cols = [x for x in valid_X.columns if x not in train_X.columns]
valid_X = valid_X.drop(drop_cols, axis=1)


#-> fill in zeros for missing columns
missing = [x for x in train_X.columns if x not in valid_X.columns]
for it in missing:
    valid_X[it]=0


#-> update order to same
valid_X = valid_X[list(train_X.columns)]

In [34]:
# need to clean up the validation set so it is consistent with the training set


#-> remove additional columns
drop_cols = [x for x in test_X.columns if x not in train_X.columns]
test_X = test_X.drop(drop_cols, axis=1)


#-> fill in zeros for missing columns
missing = [x for x in train_X.columns if x not in test_X.columns]
for it in missing:
    test_X[it]=0


#-> update order to same
test_X = test_X[list(train_X.columns)]


In [35]:
time.time()-beginning_time

101.41773724555969

In [36]:
len(train_X.columns)

823

In [37]:
sum(train_X.columns==valid_X.columns)

823

In [38]:
sum(train_X.columns==test_X.columns)

823

In [39]:
# want to save the order of the columns for future reference
itemlist=train_X.columns.tolist()

In [40]:
itemlist

['weekday',
 'hour',
 'slotwidth',
 'slotheight',
 'slotprice',
 'advertiser',
 '10006',
 '10024',
 '10031',
 '10048',
 '10052',
 '10057',
 '10059',
 '10063',
 '10067',
 '10074',
 '10075',
 '10076',
 '10077',
 '10079',
 '10083',
 '10093',
 '10102',
 '10110',
 '10111',
 '10114',
 '10115',
 '10116',
 '10117',
 '10118',
 '10120',
 '10123',
 '10125',
 '10126',
 '10127',
 '10129',
 '10130',
 '10131',
 '10133',
 '10138',
 '10140',
 '10142',
 '10145',
 '10146',
 '10147',
 '10148',
 '10149',
 '10684',
 '11092',
 '11278',
 '11379',
 '11423',
 '11512',
 '11576',
 '11632',
 '11680',
 '11724',
 '11944',
 '13042',
 '13403',
 '13496',
 '13678',
 '13776',
 '13800',
 '13866',
 '13874',
 '14273',
 '15398',
 '16593',
 '16617',
 '16661',
 '16706',
 '16751',
 '16753',
 'null',
 'useragent_android_chrome',
 'useragent_android_firefox',
 'useragent_android_ie',
 'useragent_android_maxthon',
 'useragent_android_opera',
 'useragent_android_other',
 'useragent_android_safari',
 'useragent_android_sogou',
 'use

In [41]:
import pickle

In [42]:
### save files

In [43]:
#train_X.to_pickle(os.path.join(DATA_DIR,'train_X'))
valid_X.to_pickle(os.path.join(DATA_DIR,'valid_X'))
test_X.to_pickle(os.path.join(DATA_DIR,'test_X'))

train_y.to_pickle(os.path.join(DATA_DIR,'train_y'))
valid_y.to_pickle(os.path.join(DATA_DIR,'valid_y'))

In [44]:
# train_X is too big (>4GB )so we do in two attempts
n = len(train_X)//2

train_X_1 = train_X[0:n]
train_X_2 = train_X[n:]

train_X_1.to_pickle(os.path.join(DATA_DIR,'train_X_1'))
train_X_2.to_pickle(os.path.join(DATA_DIR,'train_X_2'))

In [45]:
len(train_X) - len(train_X_1) - len(train_X_2)

0

In [46]:
# load using

#df = pd.read_pickle(file_name)


In [47]:
time.time()-beginning_time

300.80219316482544